In [1]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf

In [2]:
df = pd.read_csv('./name_gender.csv',names=['name','gender','prob'])

In [3]:
df.head()

,name,gender,prob
0,Aaban,M,1.0
1,Aabha,F,1.0
2,Aabid,M,1.0
3,Aabriella,F,1.0
4,Aada,F,1.0


In [4]:
map_letter = dict(zip(list(string.ascii_lowercase),np.arange(0,26)))

In [5]:
df['sex'] = df['gender'].map({"M":0,"F":1})

In [6]:
X,y = df['name'].values,df['sex'].values

In [7]:
def map_letter_func(s):
    return np.array([map_letter[i] for i in s.lower().strip()])

In [8]:
x = []
for names in X:
    x.append(map_letter_func(names))

In [9]:
padded_x = tf.keras.preprocessing.sequence.pad_sequences(x,maxlen=15)

In [10]:
padded_x.shape

(95025, 15)

In [11]:
X = padded_x.reshape(95025,-1,15)

In [12]:
model = tf.keras.models.Sequential(name='bilstm_model_activity')
model.add(tf.keras.layers.Input(shape= (None,padded_x[0].shape[0]),name='input'))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(512, return_sequences=True), 
    backward_layer=tf.keras.layers.LSTM(512, return_sequences=True, go_backwards=True),name = 'bilstm0'))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512),name = 'bilstm1'))
model.add(tf.keras.layers.Dense(2, activity_regularizer=tf.keras.regularizers.l2(0.002),name = 'dense_2'))
model.add(tf.keras.layers.Activation('softmax',name = 'activation'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "bilstm_model_activity"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bilstm0 (Bidirectional)     (None, None, 1024)        2162688   
                                                                 
 bilstm1 (Bidirectional)     (None, 1024)              6295552   
                                                                 
 dense_2 (Dense)             (None, 2)                 2050      
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 8,460,290
Trainable params: 8,460,290
Non-trainable params: 0
_________________________________________________________________


In [13]:
y_cat = tf.keras.utils.to_categorical(y)

In [25]:
model.fit(X,y_cat,epochs=50)

Epoch 1/10
2970/2970 [==============================] - 138s 46ms/step - loss: 0.2419 - accuracy: 0.9024
Epoch 2/10
2970/2970 [==============================] - 133s 45ms/step - loss: 0.2395 - accuracy: 0.9030
Epoch 3/10
2970/2970 [==============================] - 134s 45ms/step - loss: 0.2357 - accuracy: 0.9052
Epoch 4/10
2970/2970 [==============================] - 151s 51ms/step - loss: 0.2343 - accuracy: 0.9059
Epoch 5/10
2970/2970 [==============================] - 135s 46ms/step - loss: 0.2311 - accuracy: 0.9078
Epoch 6/10
2970/2970 [==============================] - 140s 47ms/step - loss: 0.2277 - accuracy: 0.9094
Epoch 7/10
2970/2970 [==============================] - 143s 48ms/step - loss: 0.2250 - accuracy: 0.9122
Epoch 8/10
2970/2970 [==============================] - 145s 49ms/step - loss: 0.2233 - accuracy: 0.9112
Epoch 9/10
2970/2970 [==============================] - 163s 55ms/step - loss: 0.2202 - accuracy: 0.9132
Epoch 10/10
2970/2970 [==============================] 

In [27]:
model.evaluate(X,y_cat)

2970/2970 [==============================] - 37s 12ms/step - loss: 0.2032 - accuracy: 0.9223


[0.20316444337368011, 0.9223467707633972]

In [28]:
model.save('./gender_classifier.h5')